---

In this notebook we demonstrate a method for European and Asian option pricing using the C++ SobolSeq library with Brownian Bridge approximation.

In [1]:
/ load in q scripts
\l ../code/addfunctions.q
\l ../code/brownianbridge.q
\l ../code/europeancall.q
\l ../code/loadsobol.q
\l ../code/normaldistribution.q
\l ../code/optionbrownianbirdge.q

In [2]:
/ initialize floats
/ * s = Asset at t=0
/ * k = Strike
/ * v = Volatility
/ * r = Interest rate, if Asian > .1
/ * q = Dividents
/ * t = Expiry
/ * mkt = `asian or `european
params:`s`k`v`r`q`t`mkt!(100 100 .2 .05 0 1.),`european

/ initialize integers
nstart:8      / initial number of samples
ntotal:1024   / total number of paths
nsteps:1000   / number of time steps 
ntrials:20    / number of trials for RMSE
npaths:1      / number of paths, i_N in C code

/ initialize booleans
bools:`sobol`bbridge!11b  /sobol: 1b=SobolSeq, 0b=Random, bbridge: 1b=BrownianBridge, 0b=Standard

In [ ]:
/begin timer
start:.z.t
-1"Running for "string[ntrials]," trials.";

bs:$[a:`asian~m:params`mkt;
 [-1"Asian geometric average call";asian_avgprx_discrete[nsteps]. params`s`k`v`r`q`t];
 `european~m;
 [-1"European call";bsprx . params`s`k`v`r`q`t];
 '`$"select asian or european"];

-1$[bools`bbridge;"Brownian Bridge";"Standard"]," approximation.";
-1"Number of time steps = ",string nsteps;
-1"Black-Scholes price = ",string bs;

/first loop 
N:nstart
while[N<ntotal;
 callprx:geocall[bs;params;;N;nsteps;ntrials]each([]sobol:110b;bbridge:101b);
 price :`sobol_bb`sobol_st`random_bb!callprx[;0];
 cvgerr:`sobol_bb`sobol_st`random_bb!callprx[;1];
 -1"\nConvergence error:\n";
 show flip enlist each(`npaths`lastcallprx!(N;first last callprx)),cvgerr;
 -1"\nPrice:\n";
 show flip enlist each price;
 N+:64]

-1"\nCPU time = ",string .z.t-start;

          
European call
Brownian Bridge approximation.
Number of time steps = 1000
Black-Scholes price = 10.45058

Convergence error:

npaths lastcallprx sobol_bb sobol_st random_bb
----------------------------------------------
8      2.896925    7.553651 10.45058 7.553651 

Price:

sobol_bb sobol_st random_bb
---------------------------
2.896925 0        2.896925 


---